In [1]:
import os

import numpy as np
from sklearn.preprocessing import LabelEncoder
from keras.utils import np_utils
import gc
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint
from kaggle_datasets import KaggleDatasets
import transformers
from transformers import TFAutoModel, AutoTokenizer
from tqdm.notebook import tqdm
from sklearn.model_selection import train_test_split

from sklearn.metrics import roc_auc_score, accuracy_score, f1_score
from sklearn.metrics import confusion_matrix
from tensorflow.keras.callbacks import Callback 
from tokenizers import Tokenizer, models, pre_tokenizers, decoders, processors

Using TensorFlow backend.


## Helper Functions

In [2]:
def regular_encode(texts, tokenizer, maxlen=512):
    enc_di = tokenizer.batch_encode_plus(
        texts, 
        return_attention_masks=False, 
        return_token_type_ids=False,
        pad_to_max_length=True,
        max_length=maxlen
    )
    
    return np.array(enc_di['input_ids'])

In [3]:
def build_model(transformer, max_len=512, hidden_dim=32, num_of_classes=3):
    input_word_ids = Input(shape=(max_len,), dtype=tf.int32, name="input_word_ids")
    sequence_output = transformer(input_word_ids)[0]
    cls_token = sequence_output[:, 0, :]
    
    out = Dense(num_of_classes, activation='sigmoid')(cls_token)
    
    model = Model(inputs=input_word_ids, outputs=out)
    model.compile(Adam(lr=1e-5), loss='categorical_crossentropy', metrics=['accuracy'])
    
    return model

## TPU Configs

In [4]:
# Detect hardware, return appropriate distribution strategy
try:
    # TPU detection. No parameters necessary if TPU_NAME environment variable is
    # set: this is always the case on Kaggle.
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    print('Running on TPU ', tpu.master())
except ValueError:
    tpu = None

if tpu:
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
else:
    # Default distribution strategy in Tensorflow. Works on CPU and single GPU.
    strategy = tf.distribute.get_strategy()

print("REPLICAS: ", strategy.num_replicas_in_sync)

REPLICAS:  1


In [5]:
AUTO = tf.data.experimental.AUTOTUNE

# Configuration
EPOCHS = 2
BATCH_SIZE = 16 * strategy.num_replicas_in_sync
MAX_LEN = 96
MODEL = 'bert-base-multilingual-cased'

## Create fast tokenizer

In [6]:
# First load the real tokenizer
tokenizer = AutoTokenizer.from_pretrained(MODEL)

## Load text data into memory

In [7]:
AUGMENT_DATA = True

In [8]:
dbpedia_df = pd.read_csv("/kaggle/input/test-06366/dbpedia.csv", sep="|")
faketest_dbpedia_df = dbpedia_df.sample(4400, random_state=42) # Test set from organizers has ca. 4400 examples

wikidata_df = pd.read_csv("/kaggle/input/test-06366/wikidata.csv", sep="|")

common_df = dbpedia_df.append(wikidata_df)

if AUGMENT_DATA:
    common_de_df = dbpedia_df.copy()
    common_de_df.question = dbpedia_df.question_de

    common_fr_df = dbpedia_df.copy()
    common_fr_df.question = dbpedia_df.question_fr

    common_es_df = dbpedia_df.copy()
    common_es_df.question = dbpedia_df.question_es

    common_ru_df = dbpedia_df.copy()
    common_ru_df.question = dbpedia_df.question_ru

    common_df = common_df.append(common_de_df).append(common_fr_df).append(common_es_df).append(common_ru_df)
    del common_de_df, common_fr_df, common_es_df, common_ru_df, dbpedia_df

common_df.drop(['question_de', 'question_fr', 'question_es', 'question_ru'], axis=1, inplace=True)    
common_df.dropna(subset=['question'], inplace=True)    
common_df = common_df[~common_df.id.isin(faketest_dbpedia_df.id.values)]

literal_df = common_df[common_df.category == "literal"]

X_cat_train, X_cat_test, y_cat_train, y_cat_test = train_test_split(common_df.question.astype(str), common_df.category, test_size=0.33, random_state=42)
X_lit_train, X_lit_test, y_lit_train, y_lit_test = train_test_split(literal_df.question.astype(str), literal_df.type_1, test_size=0.33, random_state=42)

In [9]:
common_df.shape

(83288, 10)

## Category classifier

In [10]:
encoder_cat = LabelEncoder()
encoder_cat.fit(common_df.category)
encoded_Y_cat_train = encoder_cat.transform(y_cat_train)
encoded_Y_cat_test = encoder_cat.transform(y_cat_test)
dummy_y_cat_train = np_utils.to_categorical(encoded_Y_cat_train) # convert integers to dummy variables (i.e. one hot encoded)

np.save('encoder_cat.npy', encoder_cat.classes_)

In [11]:
%%time 

x_cat_train = regular_encode(X_cat_train.values.astype(str), tokenizer, maxlen=MAX_LEN)
x_cat_test = regular_encode(X_cat_test.values.astype(str), tokenizer, maxlen=MAX_LEN)

CPU times: user 37.9 s, sys: 263 ms, total: 38.1 s
Wall time: 38.3 s


In [12]:
train_dataset_cat = (
    tf.data.Dataset
    .from_tensor_slices((x_cat_train, dummy_y_cat_train))
    .repeat()
    .shuffle(2048)
    .batch(BATCH_SIZE)
    .prefetch(AUTO)
)

test_dataset_cat = (
    tf.data.Dataset
    .from_tensor_slices(x_cat_test)
    .batch(BATCH_SIZE)
)

In [13]:
%%time
with strategy.scope():
    transformer_layer = TFAutoModel.from_pretrained(MODEL)
    model_cat = build_model(transformer_layer, max_len=MAX_LEN, num_of_classes=common_df.category.nunique())


CPU times: user 34.8 s, sys: 4.54 s, total: 39.4 s
Wall time: 1min 12s


In [14]:
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss', 
    verbose=1,
    patience=1,
    mode='min',
    restore_best_weights=True)

In [15]:
n_steps = x_cat_train.shape[0] // BATCH_SIZE
train_history = model_cat.fit(
    train_dataset_cat,
    steps_per_epoch=n_steps,
    epochs=EPOCHS,
    callbacks=[early_stopping]
)

model_cat.save("model_cat")

Train for 3487 steps
Epoch 1/2
3487/3487 [==============================] - 810s 232ms/step - loss: 0.2001 - accuracy: 0.9284
Epoch 2/2
3487/3487 [==============================] - 796s 228ms/step - loss: 0.0946 - accuracy: 0.9694


In [16]:
y_cat_test_pred = np.argmax(model_cat.predict(test_dataset_cat, verbose=1), axis=1)
del model_cat

1718/1718 [==============================] - 146s 85ms/step


In [17]:
acc = accuracy_score(encoded_Y_cat_test, y_cat_test_pred)
print('Accuracy: {0}'.format(acc))
print('!!!TEST set is also multilingual now!!!')

Accuracy: 0.9702030124426981
!!!TEST set is also multilingual now!!!


## Literal classifier

In [18]:
encoder_lit = LabelEncoder()
encoder_lit.fit(literal_df.type_1)
encoded_Y_lit_train = encoder_lit.transform(y_lit_train)
encoded_Y_lit_test = encoder_lit.transform(y_lit_test)
dummy_y_lit_train = np_utils.to_categorical(encoded_Y_lit_train) # convert integers to dummy variables (i.e. one hot encoded)

np.save('encoder_lit.npy', encoder_lit.classes_)

In [19]:
%%time 

x_lit_train = regular_encode(X_lit_train.values.astype(str), tokenizer, maxlen=MAX_LEN)
x_lit_test = regular_encode(X_lit_test.values.astype(str), tokenizer, maxlen=MAX_LEN)

CPU times: user 10.4 s, sys: 0 ns, total: 10.4 s
Wall time: 10.4 s


In [20]:
train_dataset_lit = (
    tf.data.Dataset
    .from_tensor_slices((x_lit_train, dummy_y_lit_train))
    .repeat()
    .shuffle(2048)
    .batch(BATCH_SIZE)
    .prefetch(AUTO)
)

test_dataset_lit = (
    tf.data.Dataset
    .from_tensor_slices(x_lit_test)
    .batch(BATCH_SIZE)
)

In [21]:
%%time
with strategy.scope():
    transformer_layer = TFAutoModel.from_pretrained(MODEL)
    model_lit = build_model(transformer_layer, max_len=MAX_LEN, num_of_classes=literal_df.type_1.nunique())

CPU times: user 3.53 s, sys: 857 ms, total: 4.39 s
Wall time: 4.85 s


In [22]:
n_steps = x_lit_train.shape[0] // BATCH_SIZE
train_history = model_lit.fit(
    train_dataset_lit,
    steps_per_epoch=n_steps,
    epochs=EPOCHS,
    callbacks=[early_stopping]
)

model_lit.save("model_lit")

Train for 1006 steps
Epoch 1/2
1006/1006 [==============================] - 246s 245ms/step - loss: 0.2423 - accuracy: 0.8982
Epoch 2/2
1006/1006 [==============================] - 231s 229ms/step - loss: 0.0766 - accuracy: 0.9747


In [23]:
y_lit_test_pred = np.argmax(model_lit.predict(test_dataset_lit, verbose=1), axis=1)
del model_lit

496/496 [==============================] - 44s 89ms/step


In [24]:
acc = accuracy_score(encoded_Y_lit_test, y_lit_test_pred)
print('Accuracy: {0}'.format(acc))
print('!!!TEST set is also multilingual now!!!')

Accuracy: 0.9682219419924338
!!!TEST set is also multilingual now!!!
